In [4]:
import sys
sys.path.insert(0,'/home/csunix/schtmt/NewFolder/caffe_Sep_anaconda/python')
import numpy as np
import matplotlib.pyplot as plt
import caffe
import h5py
import os
import time
import scipy.io
import scipy.misc

caffe.set_device(0)
caffe.set_mode_gpu()

# aug1
net = caffe.Net('test2.prototxt',
                '../convLSTM_models/avenueRecons_aug1.caffemodel', 
                caffe.TEST) 

In [6]:
import fnmatch
import os
from PIL import Image
frame_path='/usr/not-backed-up/1_DATABASE/Avenue Dataset/testing_vol/'
# aug1
save_path = '/usr/not-backed-up/1_convlstm/avenue_AE6/'
# aug2
# save_path = '/usr/not-backed-up/1_convlstm/avenue_AE_aug2/'
if not os.path.exists(save_path):
    os.mkdir(save_path)

for seq in range(21):
    print(seq)    
    # load images
#     mat = scipy.io.loadmat('file.mat')
    if seq < 9:                
        dirpath = (frame_path + 'vol0%d.mat' % (seq+1))
    else:
        dirpath = (frame_path + 'vol%d.mat' % (seq+1))
    vol = scipy.io.loadmat(dirpath)
    vol = np.array(vol['vol'])
    nfr = np.shape(vol)[2]
    
    frame_seq = []
    for fr in range(nfr):
        im = scipy.misc.imresize(vol[:,:,fr],(227,227),interp='bilinear')              
        im = np.reshape(im,[1,227,227])
        im = im.astype(float) / 255.        
        if fr == 0:
            frame_seq = im
        else:            
            frame_seq = np.append(frame_seq,im,0)
    frame_error = np.zeros([1,nfr-4])
    print(frame_error.shape)
    for fr in range(nfr-4):
        err = []
        cur_vol = frame_seq[fr:fr+5,:,:]
        gt = np.zeros(cur_vol.shape)
        for ix in range(5):
            gt[ix,:,:] = cur_vol[4-ix,:,:] 
            
        net.blobs['input'].data[...] = cur_vol.reshape([1,5,1,227,227])
        out = net.forward()        
        recons = net.blobs['deconv3'].data
        recons = np.reshape(recons,[5,227,227])
        err = np.power((gt - recons),2)
               
        # error of 5 frames
        err = np.reshape(err,[5,227*227])
        err = np.sqrt(np.sum(err,1))
        frame_error[:,fr] = np.sum(err)   
    filename = ("test_%d_error.h5" % (seq+1))       
    h5f = h5py.File(save_path + filename, 'w')
    h5f.create_dataset('frame_error', data=frame_error)                
    h5f.close()

0
(1, 1435)
1
(1, 1207)
2
(1, 919)
3
(1, 943)
4
(1, 1003)
5
(1, 1279)
6
(1, 601)
7
(1, 32)
8
(1, 1171)
9
(1, 837)
10
(1, 468)
11
(1, 1267)
12
(1, 545)
13
(1, 503)
14
(1, 997)
15
(1, 736)
16
(1, 422)
17
(1, 290)
18
(1, 244)
19
(1, 269)
20
(1, 72)
